<a href="https://colab.research.google.com/github/giuliovv/ANNDL_competition_1/blob/master/giulio_models/nasnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

colab = "True" #@param ['True','False']
if colab == "True":
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/Colab Notebooks/
  if not os.path.isdir('training'):
    !unzip dataset.zip


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab Notebooks


In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
dataset_dir = 'training'
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import shutil
if not "test" in os.listdir():
  print("No test directory!")
  for label in labels:
    print(label)
    image_names = [pic for pic in os.listdir("training/"+label)]
    train_images, test_images = train_test_split(image_names, test_size=0.05)
    if not 'test/'+label in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test/'+label+'/')
      except OSError:
        print("Failed to create a dir for ", '/test/'+label+'/')
      else:
        print("Success creating folder ", label)
    for name in test_images:
      shutil.move(os.path.abspath(os.getcwd())+'/training/'+label+'/'+name, os.path.abspath(os.getcwd())+'/test/'+label+'/'+name)
  print("Transfered all testing data!")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
BATCH_SIZE = 256
VALIDATE_BATCH_SIZE = BATCH_SIZE

train_generator = ImageDataGenerator(
## when rotating some pixels remain unfilled, to fill them the default function is to take the "nearest pixel color", this maybe screws up the learning process so I left it for later training
                                    rotation_range=90,
                                    fill_mode='nearest',
                                    brightness_range=(0.2,1.8),
                                    channel_shift_range=150,
                                    shear_range=0.2,
                                    zoom_range=0.5,
                                    width_shift_range=0.1, 
                                    height_shift_range=0.1,
                                    horizontal_flip=True, 
                                    vertical_flip=True,
                                    validation_split=0.2,
                                    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input) # VGG16 preprocessing
test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

traingen = train_generator.flow_from_directory('training',
                                              target_size=(331, 331),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='training',
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

validgen = train_generator.flow_from_directory('training',
                                              target_size=(331, 331),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='validation',
                                              batch_size=VALIDATE_BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

testgen = test_generator.flow_from_directory('test',
                                            target_size=(331, 331),
                                            class_mode='categorical',
                                            classes=labels,
                                            batch_size=1,
                                            shuffle=False,
                                            seed=42)

Found 13238 images belonging to 14 classes.
Found 3301 images belonging to 14 classes.
Found 877 images belonging to 14 classes.


In [ ]:
n_classes = len(labels)

In [ ]:
base_model = NASNetLarge(input_shape=(331, 331, 3), weights='imagenet', include_top=False, classes=n_classes)

In [ ]:
load = False

if load:
  model = tf.keras.models.load_model("nasnet_only_top")

In [ ]:
from keras.layers import LeakyReLU
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01)

if not load:
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation=lrelu, name="first")(x)
  x = Dense(512, activation=lrelu, name="second")(x)
  # and a logistic layer -- let's say we have n_classes classes
  predictions = Dense(n_classes, activation='softmax')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
if not load:

  # first: train only the top layers (which were randomly initialized)
  # i.e. freeze all convolutional InceptionV3 layers
  for layer in base_model.layers:
      layer.trainable = False

  optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

  # compile the model (should be done *after* setting layers to non-trainable)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = []

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)
    
#LRPlateau
lr_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    min_lr=0,
)  

callbacks.append(lr_plateau_callback)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

callbacks.append(tensorboard_callback)

backup = tf.keras.callbacks.experimental.BackupAndRestore(
    "top_backup_nasnet"
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "./logs"

In [ ]:
if not load:
  # train the model on the new data for a few epochs
  model.fit(traingen, epochs=8, steps_per_epoch=10, batch_size=BATCH_SIZE, validation_data=validgen, callbacks=callbacks+[backup])

In [ ]:
if not load:
  model.save("nasnet_only_top")

In [ ]:
# Freeze original layers
model.trainable = True    
set_trainable = False
for layer in model.layers:
  if layer.name == 'activation_166':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
  print("layer {} is {}".format(layer.name, '+++trainable' if layer.trainable else '---frozen'))


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
callbacks = []

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)
    
#LRPlateau
# lr_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor="val_loss",
#     factor=0.2,
#     patience=3,
#     min_lr=0,
# )  

# callbacks.append(lr_plateau_callback)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

callbacks.append(tensorboard_callback)

backup = tf.keras.callbacks.experimental.BackupAndRestore(
    "second_part_backup_nasnet"
)

In [ ]:
n_steps = traingen.samples / BATCH_SIZE
n_val_steps = validgen.samples / VALIDATE_BATCH_SIZE

model.fit(traingen, epochs=25, steps_per_epoch=10,  batch_size=BATCH_SIZE, validation_data=validgen, callbacks=callbacks+[backup])

In [ ]:
model.save("nasnet")

In [ ]:
model.evaluate(testgen)